In [1]:
from transformers import TFAutoModel ,  AutoTokenizer
from datasets import load_dataset

In [2]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

2023-12-08 22:03:56.730740: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 22:03:56.980405: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 22:03:56.981623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 22:03:58.801666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-08 22:04:01.277627: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2023-12-08 22:04:01.713980: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2023-12-08 22:

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
inputs = tokenizer(["hello world","hi how are you"] ,padding=True ,truncation=True ,return_tensors="tf")
inputs

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0,    0],
       [ 101, 7632, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [5]:
outputs = model(inputs)
outputs

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 6, 768), dtype=float32, numpy=
array([[[-0.16888388,  0.1360631 , -0.13940057, ..., -0.6251126 ,
          0.05217275,  0.3671454 ],
        [-0.3632752 ,  0.14121905,  0.879988  , ...,  0.10433033,
          0.28875685,  0.37267917],
        [-0.69859445, -0.69879746,  0.0645021 , ..., -0.22103682,
          0.00986898, -0.5939794 ],
        [ 0.8309833 ,  0.12366708, -0.15119022, ...,  0.10309598,
         -0.6779266 , -0.262852  ],
        [-0.40266657, -0.01928248,  0.5732505 , ..., -0.20656855,
          0.0233856 ,  0.20126322],
        [-0.62284076, -0.27453515,  0.18117645, ..., -0.12944834,
         -0.03839098, -0.05733182]],

       [[ 0.09286559, -0.02636376, -0.12239322, ..., -0.21063536,
          0.17386323,  0.17250998],
        [ 0.4074201 , -0.05930932,  0.5523465 , ..., -0.6790565 ,
          0.65557456, -0.29456455],
        [-0.21155256, -0.6858641 , -0.4628085 , ...,  0.1527842 

In [6]:
emotions = load_dataset("Setfit/emotion")


Repo card metadata block was not found. Setting CardData to empty.


In [7]:
def tokenize(batch):
    return tokenizer(batch["text"] ,padding=True ,truncation=True)

In [8]:
emotions_encoded = emotions.map(tokenize ,batched=True ,batch_size=None)

In [9]:
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [10]:
import tensorflow as tf

emotions_encoded.set_format('tf', 
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

BATCH_SIZE = 64

def order(inp):
    
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    },data[0]

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded["train"][:])
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
train_dataset = train_dataset.map(order ,num_parallel_calls=tf.data.AUTOTUNE )


test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded["test"][:])
test_dataset = test_dataset.batch(BATCH_SIZE).shuffle(1000)
test_dataset = test_dataset.map(order ,num_parallel_calls=tf.data.AUTOTUNE )

In [12]:
inp , out = next(iter(train_dataset))

In [13]:
inp

{'input_ids': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
 array([[  101,  1045,  2123, ...,     0,     0,     0],
        [  101,  1045,  2514, ...,     0,     0,     0],
        [  101,  1045,  3335, ...,     0,     0,     0],
        ...,
        [  101, 10047,  3110, ...,     0,     0,     0],
        [  101, 10047,  9069, ...,     0,     0,     0],
        [  101,  1045,  2514, ...,     0,     0,     0]])>,
 'attention_mask': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>,
 'token_type_ids': <tf.Tensor: shape=(64, 87), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>}

In [14]:
out

<tf.Tensor: shape=(64,), dtype=int64, numpy=
array([3, 0, 1, 0, 0, 1, 4, 0, 0, 2, 1, 1, 1, 1, 0, 2, 4, 2, 0, 4, 0, 0,
       0, 3, 0, 4, 1, 0, 0, 1, 1, 0, 1, 4, 0, 1, 0, 0, 2, 0, 1, 1, 3, 0,
       0, 1, 1, 4, 0, 1, 1, 1, 4, 0, 0, 0, 1, 1, 4, 0, 1, 4, 3, 1])>

In [15]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert= bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation="softmax")
        
    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [16]:
classifier = BERTForClassification(model ,num_classes=6)

classifier.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [18]:
history = classifier.fit(
    train_dataset ,
    epochs=3
)

Epoch 1/3


: 